In [105]:
# Assumptions 
# - 2x farm where one asset is stablecoin, e.g.  ETH/USDC
# - there is always borrow availability
# - borrow rate, trading fees, rewards are all constant


from math import exp, sqrt
from statistics import mean
import pandas as pd 


class Agent():
    
    def __init__(self, equity, price, borrow_rate, farming_rate):
        print("init")
        
        self.last_price = price
        self.borrow_rate = borrow_rate
        self.farming_rate = farming_rate
        
        # Make an intial investment - 50% stablecoin, 50% borrowed token
        self.equity = equity
        self.assetA_count = equity / price  # token
        self.assetB_count = equity          # stablecoin
        self.debt_value =  self.assetA_count * price

        # Settings for naive rebalancer - trigger when price moves 10% in either direction
        self.upper = price * 1.1
        self.lower = price * 0.9
        
        self.last_rebalance_step = 0
        self.last_rebalance_price = price
        
    def rebalance(self):
        print("rebalance")
        
        # update steps since rebalance
        # update self.unrealised_position_value
        # update self.unrealised_debt_value
        # update self.unrealised_equity
    
    # Calculates since last rebalance, assume that gains / losses only crystalised on rebalance
    def calculate_position(self, step, new_price):
        print("calculate_pnl")
        steps_since_rebalance = step - self.last_rebalance_step

        price_change_ratio = new_price / self.last_rebalance_price
        lp_multiplier = sqrt(price_change_ratio)
        
        # Taken from https://docs.google.com/spreadsheets/d/1mGZDKgtRNXilSQT4U5YbqNPCSqMiPayuqvD8Xx4Keqo/
        position_value = (2*self.equity*lp_multiplier) + 2*mean([self.equity*lp_multiplier, self.equity])*(exp(self.farming_rate*steps_since_rebalance)-1)
        
        debt_value = self.assetA_count * new_price * exp(self.borrow_rate*steps_since_rebalance)
        
        equity = position_value - debt_value
        
        self.unrealised_position_value = position_value
        self.unrealised_debt_value = debt_value
        self.unrealised_equity = equity

        print ("steps_since_rebalance=" + str(steps_since_rebalance))
        print ("price_change_ratio=" + str(price_change_ratio))
        print ("lp_multiplier=" + str(lp_multiplier))
        print ("position_value=" + str(self.unrealised_position_value))
        print ("debt_value=" + str(self.unrealised_debt_value))
        print ("equity_value=" + str(self.unrealised_equity))

    def action(self, step, new_price):
        print ("\n")
        print ("ACTION...  price is " + str(new_price))
        
        self.calculate_position(step, new_price)
        self.last_price = new_price
        
     #    Rebalance if necessary
     #   if ((price > self.upper) or (price < self.lower)):
     #       self.rebalance()
     #       self.calculate_position()
            
    
        

# RUN SIMULATION
df = pd.DataFrame(columns = ['step', 'price', 'assetA_count', 'assetB_count', 'unrealised_position_value', 'unrealised_debt_value', 'unrealised_equity_value', 'last_rebalanced'])
price = [13.70, 13.70, 14.385, 15.07, 15.76, 11.65, 12.33]

INITIAL_EQUITY = 10000
BORROW_APR = 85        # token borrow rate (%)
FARMING_APR = 100      # farming and all rewards rate (%)


# Assume a time step is 1 day
agent = Agent(
    INITIAL_EQUITY,        # initial equity
    price.pop(0),          # initial token_price 
    BORROW_APR/365/100,    # borrow rate per time step 
    FARMING_APR/365/100    # farming rate at 1x leverage per time step
)

step = 1
for p in price:
    agent.action(step, p) 
    
    df = df.append ({
        'step': step,
        'price': p,
        'assetA_count' : agent.assetA_count,
        'assetB_count' : agent.assetB_count,
        'unrealised_position_value': agent.unrealised_position_value,
        'unrealised_debt_value': agent.unrealised_debt_value,
        'unrealised_equity': agent.unrealised_equity,
        'last_rebalanced': agent.last_rebalance_step
    },
        ignore_index=True)
    
    step += 1


df.head()


# UNUSED
# entry_fee = 0
# exit_fee = 0
# controller_fee = 0.001  
# reinvestent_fee = 0.039 
# tx_fee = 0  # network fee
# performance_fee = 0



#TODO
# repeated iterations of action - calcs are working for 1 step but need to update state - OR only update state on rebalance when you crystalise loss, at other times keep track of timesteps since last rebalance to calculate equity etc.
# put in real price data
# do rebalances
# deal with nuances - e.g. francium autocompounds rewards every 15m, compounding of borrow?  https://docs.francium.io/product/protocol-parameters
            


init


ACTION...  price is 13.7
calculate_pnl
steps_since_rebalance=1
price_change_ratio=1.0
lp_multiplier=1.0
position_value=20054.869650130822
debt_value=10023.314808075489
equity_value=10031.554842055333


ACTION...  price is 14.385
calculate_pnl
steps_since_rebalance=2
price_change_ratio=1.05
lp_multiplier=1.02469507659596
position_value=20605.1482350423
debt_value=10549.018172887465
equity_value=10056.130062154834


ACTION...  price is 15.07
calculate_pnl
steps_since_rebalance=3
price_change_ratio=1.1
lp_multiplier=1.0488088481701516
position_value=21145.266146352056
debt_value=11077.118387548231
equity_value=10068.147758803825


ACTION...  price is 15.76
calculate_pnl
steps_since_rebalance=4
price_change_ratio=1.1503649635036497
lp_multiplier=1.072550681088614
position_value=21679.39156388769
debt_value=11611.307561326792
equity_value=10068.084002560898


ACTION...  price is 11.65
calculate_pnl
steps_since_rebalance=5
price_change_ratio=0.8503649635036497
lp_multiplier=0.92215235

,step,price,assetA_count,assetB_count,unrealised_position_value,unrealised_debt_value,unrealised_equity_value,last_rebalanced,unrealised_equity
0,1.0,13.700,729.927007,10000.0,20054.869650,10023.314808,NaN,0.0,10031.554842
1,2.0,14.385,729.927007,10000.0,20605.148235,10549.018173,NaN,0.0,10056.130062
2,3.0,15.070,729.927007,10000.0,21145.266146,11077.118388,NaN,0.0,10068.147759
3,4.0,15.760,729.927007,10000.0,21679.391564,11611.307561,NaN,0.0,10068.084003
4,5.0,11.650,729.927007,10000.0,18708.167363,8603.243435,NaN,0.0,10104.923928
